In [ ]:
%cd C:/Users/alapa/funiegan/

In [ ]:
import argparse
from torchvision import transforms
import tkinter as tk
from tkinter import filedialog
import torch
from PIL import Image
from torch.autograd import Variable
from IPython.display import display
from os.path import exists

print("upload image")
# Create a Tkinter window
root = tk.Tk()
# Hide the main window

# Prompt the user to select an image file
file_path = filedialog.askopenfilename(
    filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.gif;*.bmp")]
)

if not file_path:
    print("No file selected. Exiting.")
    exit()

# Load the selected image
img = Image.open(file_path)
display(img)
root.withdraw()

print("image uploaded")
img_width, img_height, channels = 256, 256, 3
transforms_ = [transforms.Resize((img_height, img_width), Image.BICUBIC),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
transform = transforms.Compose(transforms_)

path = "EUVP/test_samples/single_result"

for i in range(0, 11):
    generator = f"checkpoints/FunieGAN/EUVP/generator_{i}.pth"

    # Argument parsing (if needed)
    parser = argparse.ArgumentParser()
    parser.add_argument("--model_name", type=str, default="funiegan")  # or "ugan"
    parser.add_argument("--model_path", type=str, default=generator)
    opt, unknown = parser.parse_known_args()

    print(opt.model_path)

    # Check if model exists
    assert exists(opt.model_path), "model not found"
    is_cuda = torch.cuda.is_available()
    Tensor = torch.cuda.FloatTensor if is_cuda else torch.FloatTensor 

    if opt.model_name.lower()=='funiegan':
        from PyTorch.nets import funiegan
        model = funiegan.GeneratorFunieGAN()
    elif opt.model_name.lower()=='ugan':
        from nets.ugan import UGAN_Nets
        model = UGAN_Nets(base_model='pix2pix').netG
    else: 
        # other models
        continue

    model.load_state_dict(torch.load(opt.model_path))
    if is_cuda: model.cuda()
    model.eval()
    print ("Loaded model from %s" % (opt.model_path))
    
    inp_img = transform(img)
    inp_img = Variable(inp_img).type(Tensor).unsqueeze(0)
    
    # Generate enhanced image
    with torch.no_grad():
        gen_img = model(inp_img)

    # Convert tensors to PIL images for display
    inp_img_pil = transforms.ToPILImage()(inp_img.squeeze(0).cpu())
    gen_img_pil = transforms.ToPILImage()(gen_img.squeeze(0).cpu())

    # Display input and generated images side by side
    display(inp_img_pil)
    display(gen_img_pil)


In [ ]:
import argparse
from torchvision import transforms
import tkinter as tk
from tkinter import filedialog
import torch
from PIL import Image
from torch.autograd import Variable
from IPython.display import display
from torchvision.utils import save_image
from os.path import exists

print("upload image")
# Create a Tkinter window
root = tk.Tk()
# Hide the main window

# Prompt the user to select an image file
file_path = filedialog.askopenfilename(
    filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.gif;*.bmp")]
)

if not file_path:
    print("No file selected. Exiting.")
    exit()

# Load the selected image
img = Image.open(file_path)
display(img)
root.withdraw()

print("image uploaded")
img_width, img_height, channels = 256, 256, 3
transforms_ = [transforms.Resize((img_height, img_width), Image.BICUBIC),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]
transform = transforms.Compose(transforms_)

path = "EUVP/test_samples/single_result"

for i in range(0, 11):
    generator = f"checkpoints/FunieGAN/EUVP/generator_{i}.pth"

    # Argument parsing (if needed)
    parser = argparse.ArgumentParser()
    parser.add_argument("--model_name", type=str, default="funiegan")  # or "ugan"
    parser.add_argument("--model_path", type=str, default=generator)
    opt, unknown = parser.parse_known_args()

    print(opt.model_path)

    # Check if model exists
    assert exists(opt.model_path), "model not found"
    is_cuda = torch.cuda.is_available()
    Tensor = torch.cuda.FloatTensor if is_cuda else torch.FloatTensor 

    if opt.model_name.lower()=='funiegan':
        from PyTorch.nets import funiegan
        model = funiegan.GeneratorFunieGAN()
    elif opt.model_name.lower()=='ugan':
        from nets.ugan import UGAN_Nets
        model = UGAN_Nets(base_model='pix2pix').netG
    else: 
        # other models
        continue

    model.load_state_dict(torch.load(opt.model_path))
    if is_cuda: model.cuda()
    model.eval()
    print ("Loaded model from %s" % (opt.model_path))
    
 
    inp_img = transform(img) 
    inp_img = Variable(inp_img).type(Tensor).unsqueeze(0)
    
    # Generate enhanced image
    with torch.no_grad():
        gen_img = model(inp_img)

    img_sample = torch.cat((inp_img.data, gen_img.data), -1)
    display(img_sample)